In [66]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.7.1


In [67]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [68]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [69]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [70]:
from operator import truediv
from tabnanny import verbose


checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

In [71]:
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[checkpoint_cb]) 

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 1.1739 - sparse_categorical_accuracy: 0.6470
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 12ms/step - loss: 1.1739 - sparse_categorical_accuracy: 0.6470 - val_loss: 0.7266 - val_sparse_categorical_accuracy: 0.7820
Epoch 2/10
27/32 [========================>.....] - ETA: 0s - loss: 0.4310 - sparse_categorical_accuracy: 0.8843
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4219 - sparse_categorical_accuracy: 0.8910 - val_loss: 0.5158 - val_sparse_categorical_accuracy: 0.8470
Epoch 3/10
28/32 [=========================>....] - ETA: 0s - loss: 0.2866 - sparse_categorical_accuracy: 0.9219
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2783 - sparse_categorical_accuracy: 0.9240 - val_loss: 0.4594 - val_sparse_categorical_accuracy: 0.8560
Epoch 4/10
19/3

In [72]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [73]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3682 - sparse_categorical_accuracy: 0.1090 - 140ms/epoch - 4ms/step
Untrained model, accuracy: 10.90%


In [74]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [75]:
os.listdir(checkpoint_dir)

['cp-0030.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'checkpoint',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001']

In [76]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [77]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4742 - sparse_categorical_accuracy: 0.8780 - 140ms/epoch - 4ms/step
Restored model, accuracy: 87.80%


In [78]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4742 - sparse_categorical_accuracy: 0.8780 - 137ms/epoch - 4ms/step
Restored model, accuracy: 87.80%


In [79]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1283 - sparse_categorical_accuracy: 0.6940
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4270 - sparse_categorical_accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2824 - sparse_categorical_accuracy: 0.9310
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2216 - sparse_categorical_accuracy: 0.9360
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1531 - sparse_categorical_accuracy: 0.9700
INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [80]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [81]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4495 - sparse_categorical_accuracy: 0.8580 - 144ms/epoch - 5ms/step
Restored model, accuracy: 85.80%
(1000, 10)


In [82]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1615 - sparse_categorical_accuracy: 0.6830
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4354 - sparse_categorical_accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2951 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2143 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1530 - sparse_categorical_accuracy: 0.9630


In [83]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [84]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4087 - sparse_categorical_accuracy: 0.8750 - 138ms/epoch - 4ms/step
Restored model, accuracy: 87.50%
